In [1]:
import numpy as np 
import pandas as pd 
from cryptography.fernet import Fernet
from datetime import datetime, date, timedelta
from tqdm import tqdm_notebook as tqdm
import os
import pymssql as mssql
import calendar
import time
from multiprocessing import Pool
from functools import partial

import ALL_STOCK_preprocess_function_label as func
import VWAP_feature_function_rolling_week as func1

In [4]:
d1 = date(2019, 10, 1)
d2 = date(2019, 10, 31)

In [8]:
delta = d2 - d1         # timedelta
date_list = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
total_list = []

for _, target_date in enumerate(tqdm(date_list)):
    
    if target_date.weekday() in [5, 6]:
        continue
    
    d = func.stock_query(target_date)
    stock, index, industry_index = d[0], d[1], d[2]
    df_list = [group[1] for group in stock.groupby(stock['StockNo'])]
    timedf = pd.DataFrame(d[0]['ts'].unique(), columns=['ts'])
    timedf = timedf.sort_values(by='ts')
    output_list = []
    
    if __name__ == '__main__':
        with Pool(processes=5) as pool:
            for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func.FillMissingTime, timedf=timedf, target_date=target_date), df_list), total=len(df_list)), 1):
                    if x[0]:
                        output_list.append(x[1])
                    else:
                        continue
    
    if len(output_list) == 0:
        continue
        
    df = pd.concat(output_list)
    
    df_list = [group[1] for group in df.groupby(df['StockNo'])]
    output_list = []

    if __name__ == '__main__':
        with Pool(processes=5) as pool:
            for ___, x in enumerate(tqdm(pool.imap_unordered(partial(func.merge_index, index=index, industry_index=industry_index), df_list), total=len(df_list)), 1):
                    output_list.append(x)

    df = pd.concat(output_list)
    
    df_list = [group[1] for group in df.groupby(df['StockNo'])]
    output_list = []

    if __name__ == '__main__':
        with Pool(processes=5) as pool:
            for ___, x in enumerate(tqdm(pool.imap_unordered(func1.get_label, df_list), total=len(df_list)), 1):
                output_list.append(x)

    
    df = pd.concat(output_list, axis=0)    
    df_last = df[df.ts.dt.date == target_date]

    total_list.append(df_last)

In [9]:
df = pd.concat(total_list, axis=0)

In [12]:
d = df[(df.ts.dt.date < date(2019,11,1)) & (df.ts.dt.date >= date(2019,10,1))]
d.to_csv('D:\\庫存健診開發\\data\\labeled_all.csv', index=False)

In [11]:
df['ts'].min()

Timestamp('2019-10-01 00:00:00')

In [10]:
len(df)

19267

In [11]:
df.columns

Index(['ts', 'StockNo', 'StockName', 'open', 'high', 'low', 'close', 'vol',
       'total', 'capital', 'foreign_buy', 'investment_buy', 'dealer_buy',
       'foreign_ratio', 'investment_ratio', 'dealer_ratio',
       'corporation_ratio', 'industry', 'On_Date', 'Restart_date', 'PctRank',
       'eliminate', 'VWAP', 'index_open', 'index_high', 'index_low',
       'index_close', 'index_vol', 'industry_open', 'industry_high',
       'industry_low', 'industry_close', 'industry_vol', 'VWAP_day5',
       'VWAP_after', 'Y'],
      dtype='object')